Subdiv2D
===

`Subdiv2D` 是 OpenCV 库中的一个类，用于实现二维平面的细分。细分是一种数据结构，用于表示多边形和三角形网格，并且可以用于高效的几何查询和处理，例如点定位、最近点查找、区域填充等。

以下是 `Subdiv2D` 类的一些关键特性和用法：

1. **多边形表示**：`Subdiv2D` 可以表示任意多边形，包括凸多边形和凹多边形。
2. **顶点和边**：细分由顶点和边组成，每个顶点连接到其他顶点形成边。
3. **递归四叉树**：内部使用递归四叉树数据结构来组织顶点和边，从而实现高效的查询。
4. **点定位**：可以快速找到点所在的多边形区域，这对于许多几何算法非常重要。
5. **最近点查找**：可以找到给定点最近的边界点。
6. **绘制**：可以将细分绘制到图像上，显示多边形和边界。
7. **区域填充**：可以用来实现扫描线填充算法，填充任意多边形区域。


In [4]:
#r "nuget: OpenCvSharp4.Windows"
using OpenCvSharp;

Installed Packages OpenCvSharp4.Windows, 4.10.0.20240616

In [5]:
const int Size = 600;

// 1. 常量定义和随机点生成
var rand = new Random();
var points = Enumerable.Range(0, 100).Select(_ =>
    new Point2f(rand.Next(0, Size), rand.Next(0, Size))).ToArray();

// 2. 图像初始化和绘制随机点
var imgExpr = Mat.Zeros(Size, Size, MatType.CV_8UC3);
var img = imgExpr.ToMat();
foreach (var p in points)
{
    img.Circle((Point)p, 4, Scalar.Red, -1);
}

// 3. 初始化Subdiv2D并插入点
var subdiv = new Subdiv2D();
subdiv.InitDelaunay(new Rect(0, 0, Size, Size));
subdiv.Insert(points);

Voronoi图是一种几何结构，它将平面划分为若干个区域，每个区域对应一个特定的点，称为种子点。每个区域内的所有点到该种子点的距离都比到其他种子点的距离更近。  
Voronoi图在计算几何、地理信息系统、机器人路径规划等领域有广泛的应用。  
Voronoi图的基本概念
1. 种子点（Seed Points）： 这些是用于生成Voronoi图的初始点。每个种子点都有一个对应的Voronoi区域。
2. Voronoi区域（Voronoi Cells）： 每个Voronoi区域包含所有到其对应种子点距离最近的点。区域的边界由到两个或多个种子点距离相等的点组成。
3. Voronoi边界（Voronoi Edges）： 这些是Voronoi区域之间的边界。每条边界是到两个种子点距离相等的点的集合。

In [6]:
// 4. 绘制Voronoi图
subdiv.GetVoronoiFacetList(null, out var facetList, out var facetCenters);
var vonoroi = img.Clone();
foreach (var list in facetList)
{
    var before = list.Last();
    foreach (var p in list)
    {
        vonoroi.Line((Point)before, (Point)p, new Scalar(64, 255, 128), 1);
        before = p;
    }
}

Delaunay三角剖分是一种几何结构，它将一组点划分为若干个三角形，使得没有任何点位于任何三角形的外接圆内。  
Delaunay三角剖分在计算几何、图像处理、地理信息系统等领域有广泛的应用。  
Delaunay三角剖分的基本概念
1. 点集（Point Set）： 这些是用于生成Delaunay三角剖分的初始点。
2. 三角形（Triangles）： 由点集生成的三角形，每个三角形的顶点都是点集中的点。
3. 外接圆（Circumcircle）： 每个三角形都有一个外接圆，外接圆是通过三角形的三个顶点唯一确定的圆。
4. Delaunay性质： 对于Delaunay三角剖分中的每个三角形，其外接圆内不包含任何其他点集中的点。

In [7]:
//5. 绘制Delaunay三角剖分
Vec4f[] edgeList = subdiv.GetEdgeList();
var delaunay = img.Clone();
foreach (var edge in edgeList)
{
    var p1 = new Point(edge.Item0, edge.Item1);
    var p2 = new Point(edge.Item2, edge.Item3);
    delaunay.Line(p1, p2, new Scalar(64, 255, 128), 1);
}

Cv2.ImShow("voronoi", vonoroi);
Cv2.ImShow("delaunay", delaunay);
Cv2.WaitKey();
Cv2.DestroyAllWindows();